In [ ]:
# pip install git+https://github.com/BioroboticsLab/IBA

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/BioroboticsLab/IBA to /tmp/pip-req-build-pfbdfzgp
  Running command git clone -q https://github.com/BioroboticsLab/IBA /tmp/pip-req-build-pfbdfzgp
ERROR: Operation cancelled by user


In [ ]:
# to set you cuda device
%env CUDA_VISIBLE_DEVICES=0

%load_ext autoreload
%autoreload 2

import IBA
from IBA.pytorch import IBA, tensor_to_np_img
from IBA.utils import plot_saliency_map
from IBA.pytorch import IBA, tensor_to_np_img
from visualize import *

import torch
import torchvision.models 
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize, Normalize
import matplotlib.pyplot as plt
import cv2

import os
from tqdm import tqdm_notebook
import json
from PIL import Image
import numpy as np

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
imagenet_dir = "t"

dev = torch.device('cuda:0')

model = torchvision.models.vgg16(pretrained=True)
model.to(dev).eval()

image_size = 224
    
valset = ImageFolder(
    os.path.join(imagenet_dir),
    transform=Compose([
        CenterCrop(256), Resize(image_size), ToTensor(), 
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))

trainset = ImageFolder(
    os.path.join(imagenet_dir),
    transform=Compose([
        CenterCrop(256), Resize(image_size), ToTensor(), 
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=1)
img, target = valset[0]

In [ ]:
imagePath = []
targets = [58,222,332,417,429,429,525,549,683,889,990]
features_ls = [30,28,26,24,21,19,17,14,12,10,7,5,2,0]

untrained = models.vgg16(pretrained=False)

In [ ]:
iba = IBA(model.features[17])

iba.reset_estimate()
iba.estimate(model, trainloader, device=dev, n_samples=10000, progbar=True)

In [ ]:
for layer in features_ls:
  if layer == 28:
      model.classifier = untrained.classifier
  if layer < 30:
      model.features[layer] = untrained.features[layer]

  for i, path in enumerate(imagePath):
    img = cv2.imread(path)
    target = targets[i]

    tts = ToTensor()
    input_tensor = tts(img).cuda()
    
    model = model.cuda()
    model_loss_closure = lambda x: -torch.log_softmax(model(x), 1)[:, target].mean()
    heatmap = iba.analyze(input_tensor[None].to(dev), model_loss_closure)    

    save_path = ''
    save_result(heatmap, save_path)

compute SSIM score

In [ ]:
base_path = ''

ssim_ls = compute_ssim(imagePath, base_path) 

In [ ]:
plot_ssim(ssim_ls, 'IBA', '')

In [ ]:
scorePath = '/content/drive/MyDrive/scoreCAM/sanityCheck/'
scorecam = compute_ssim(imagePath, scorePath)

gradPath = '/content/drive/MyDrive/gradCAM/sanityCheck/'
gradcam = compute_ssim(imagePath, gradPath)

risePath = '/content/drive/MyDrive/RISE/sanityCheck/'
rise = compute_ssim(imagePath, risePath)

extPath = '/content/drive/MyDrive/extremal perturbations/sanityCheck/'
ext = compute_ssim(imagePath, extPath)

In [ ]:

x = ['30','28','26','24','21','19','17','14','12','10','7','5','2','0']
labels = ['original', 'conv5-3', 'conv5-2', 'conv5-1', 'conv4-3', 'conv4-2', 'conv4-1', 'conv3-3', 'conv3-2',
              'conv3-1', 'conv2-2', 'conv2-1', 'conv1-2', 'conv1-1']
dummy = [30 * 13 / 13, 30 * 12 / 13, 30 * 11 / 13, 30 * 10 / 13, 30 * 9 / 13, 30 * 8 / 13, 30 * 7 / 13, 30 * 6 / 13, 30 * 5 / 13, 30 * 4 / 13, 30 * 3 / 13, 30 * 2 / 13, 30 * 1 / 13, 0]

mean_ssim = np.array(ssim_ls).mean(axis=0)
dummy_np = np.array(dummy)
plt.figure(figsize=(9,6))
plt.xticks(dummy, labels, rotation=40)
plt.yticks([1.0, 0.9, 0.8, 0.7, 0.6])
plt.gca().invert_xaxis()
plt.plot(dummy_np, np.array(ssim_ls).mean(axis=0), marker='D', label="IBA")
plt.plot(dummy_np, np.array(gradcam).mean(axis=0), marker='D', label="GradCAM")
plt.plot(dummy_np, np.array(scorecam).mean(axis=0), marker='D', label="ScoreCAM")
plt.plot(dummy_np, np.array(rise).mean(axis=0), marker='D', label="RISE")
plt.plot(dummy_np, np.array(ext).mean(axis=0), marker='D', label="Extremal\nPerturbation")
plt.title('VGG-16')
plt.ylabel('SSIM score')
plt.xlabel('features randomized')
plt.legend()
plt.savefig('/content/drive/MyDrive/sanity_check_whole.jpg', bbox_inches = 'tight')
# plt.show()